In [1]:
import pandas as pd
import pickle
import os
from tqdm import tqdm

In [2]:
# 1. match_info 데이터 로드
csv_file_path = './open_track1/match_info.csv' 

print("1. 데이터 파일을 읽어오는 중입니다...")
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    print(f"   -> 로드 완료! 총 {len(df)}개의 경기 데이터가 있습니다.")
else:
    print(f"Error: {csv_file_path} 파일을 찾을 수 없습니다.")
    exit() # 파일이 없으면 여기서 종료

# 2. 데이터 처리 (팀 정보 & 매치 정보)
# 진행률을 보여주기 위해 tqdm을 사용합니다.
team_dict = {}
match_dict = {}

print("\n2. 데이터를 순회하며 딕셔너리를 생성합니다...")

# iterrows()는 데이터가 많으면 느릴 수 있어, tqdm으로 진행상황을 봅니다.
# total=len(df)를 넣어줘야 진행률 바가 정확히 나옵니다.
for _, row in tqdm(df.iterrows(), total=len(df), desc="처리 중"):
    # --- [정보 추출] ---
    g_id = row['game_id']
    
    h_id = row['home_team_id']
    h_name = row['home_team_name_ko']
    venue = row['venue']
    
    a_id = row['away_team_id']
    a_name = row['away_team_name_ko']
    
    # --- [Team Dict 구축] ---
    # 홈팀 저장
    team_dict[h_id] = {
        'name': h_name,
        'venue': venue
    }
    
    # 어웨이팀 저장 (없을 경우에만)
    if a_id not in team_dict:
        team_dict[a_id] = {
            'name': a_name,
            'venue': None 
        }
        
    # --- [Match Dict 구축] ---
    # 두 번 돌 필요 없이 한 번 돌 때 같이 처리하면 훨씬 빠릅니다.
    match_dict[g_id] = {
        'home_team_id': h_id,
        'away_team_id': a_id
    }

# 3. 결과 확인 및 저장
print("\n3. 처리가 완료되었습니다.")
print(f"   - 추출된 팀 개수: {len(team_dict)}")
print(f"   - 추출된 경기 개수: {len(match_dict)}")

# 저장 단계
save_path = 'processed_match_data.pkl'
print(f"\n4. '{save_path}' 파일로 저장 중입니다...")

with open(save_path, 'wb') as f:
    pickle.dump({'match_dict': match_dict, 'team_dict': team_dict}, f)
    
print("   -> 저장 완료! 모든 작업이 끝났습니다.")

1. 데이터 파일을 읽어오는 중입니다...
   -> 로드 완료! 총 228개의 경기 데이터가 있습니다.

2. 데이터를 순회하며 딕셔너리를 생성합니다...


처리 중: 100%|██████████| 228/228 [00:00<00:00, 4949.52it/s]


3. 처리가 완료되었습니다.
   - 추출된 팀 개수: 12
   - 추출된 경기 개수: 228

4. 'processed_match_data.pkl' 파일로 저장 중입니다...
   -> 저장 완료! 모든 작업이 끝났습니다.


In [ ]:
import os
import pickle

# --- 검증 코드 시작 ---
print("\n[검증 시작] 저장된 pickle 파일을 불러와서 데이터를 확인합니다.")

if os.path.exists('match_data.pkl'):
    try:
        with open('match_data.pkl', 'rb') as f:
            loaded_data = pickle.load(f)
            
        print("✅ 파일 로드 성공!")
        
        # 데이터 구조 확인
        l_match = loaded_data.get('match_dict', {})
        l_team = loaded_data.get('team_dict', {})
        
        print(f"- 불러온 경기 수: {len(l_match)}개")
        print(f"- 불러온 팀 수: {len(l_team)}개")
        
        # 샘플 데이터 출력 (첫 번째 아이템)
        if l_match:
            first_game_id = next(iter(l_match))
            print(f"- 샘플 경기 데이터 (ID: {first_game_id}): {l_match[first_game_id]}")
            
            # 연결된 팀 정보 확인
            home_team_id = l_match[first_game_id]['home_team_id']
            if home_team_id in l_team:
                print(f"- 샘플 팀 데이터 (ID: {home_team_id}): {l_team[home_team_id]}")
            else:
                print(f"⚠️ 경고: 팀 ID {home_team_id}에 대한 정보가 team_dict에 없습니다.")
        
        print("\n🎉 모든 과정이 정상적으로 완료되었습니다.")
        
    except Exception as e:
        print(f"❌ 파일을 불러오는 중 에러가 발생했습니다: {e}")
else:
    print("❌ 'processed_match_data.pkl' 파일이 생성되지 않았습니다. 위 코드를 먼저 실행해주세요.")


[검증 시작] 저장된 pickle 파일을 불러와서 데이터를 확인합니다.
✅ 파일 로드 성공!
- 불러온 경기 수: 228개
- 불러온 팀 수: 12개
- 샘플 경기 데이터 (ID: 126283): {'home_team_id': 2354, 'away_team_id': 4639}
- 샘플 팀 데이터 (ID: 2354): {'name': '울산 HD FC', 'venue': '문수월드컵경기장'}

🎉 모든 과정이 정상적으로 완료되었습니다.
